# Prototype Model: Deteksi Code-Switching dengan Weak Supervision & Naive Bayes

Notebook ini merupakan prototype untuk mendeteksi **Code-Switching** (percampuran bahasa Indonesia dan Inggris) pada tweet menggunakan pendekatan:
1. **Weak Supervision** - Pelabelan data otomatis berbasis aturan (lexicon)
2. **Naive Bayes + TF-IDF** - Model klasifikasi teks

**Output:**
- `dataset_hasil_pelabelan.csv` - Dataset dengan label bahasa
- `confusion_matrix.png` - Visualisasi akurasi model

---
## Step 1: Import Library

Pada langkah ini, kita mengimpor semua library yang diperlukan:

| Library | Fungsi |
|---------|--------|
| `pandas` | Manipulasi data tabular (DataFrame) |
| `numpy` | Operasi numerik |
| `re` | Regular Expression untuk text cleaning |
| `matplotlib` & `seaborn` | Visualisasi data (confusion matrix) |
| `sklearn` | Machine Learning (TF-IDF, Naive Bayes, evaluasi) |

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("[✓] Semua library berhasil diimpor!")

---
## Step 2: Fungsi Pembantu untuk Output

Fungsi-fungsi ini digunakan untuk membuat output yang rapi dan mudah dibaca di terminal/console.

In [ ]:
def print_header(title):
    """Membuat header dengan border untuk judul section"""
    print("\n" + "="*70)
    print(f" {title} ".center(70, "="))
    print("="*70)

def print_separator():
    """Membuat garis pemisah"""
    print("-" * 70)

---
# TAHAP 1: WEAK SUPERVISION (PELABELAN OTOMATIS)

Weak Supervision adalah teknik pelabelan data secara otomatis menggunakan **aturan heuristik** (rules) tanpa perlu anotasi manual. Pada kasus ini, kita menggunakan **lexicon-based approach** untuk mengidentifikasi bahasa.

---
## Step 3: Muat Dataset Mentah

Dataset yang digunakan adalah `codeswitch_emotion.csv` yang berisi tweet dengan campuran bahasa Indonesia dan Inggris.

**Catatan untuk Google Colab:** Upload file dataset terlebih dahulu.

In [ ]:
# (Opsional) Upload file di Google Colab
# from google.colab import files
# uploaded = files.upload()

In [ ]:
print_header("TAHAP 1: WEAK SUPERVISION (PELABELAN DATA OTOMATIS)")

try:
    df = pd.read_csv('codeswitch_emotion.csv', on_bad_lines='skip')
    print(f"[INFO] Dataset 'codeswitch_emotion.csv' berhasil dimuat.")
    print(f"[INFO] Total data mentah: {len(df)} baris.")
    display(df.head())
except FileNotFoundError:
    print("[ERROR] File dataset tidak ditemukan. Menggunakan data dummy.")
    df = pd.DataFrame({'tweet': [
        "Aku stuck banget sama deadline tugas", 
        "I love you so much", 
        "Makan nasi goreng enak di kantin", 
        "Which is sebenernya dia fine aja"
    ]})
    display(df)

---
## Step 4: Definisi Lexicon (Kamus Kata)

Lexicon adalah kumpulan kata-kata kunci yang menjadi **penanda bahasa**. Kita mendefinisikan dua lexicon:

1. **`indo_markers`** - Kata-kata penanda Bahasa Indonesia (termasuk slang seperti `gak`, `bgt`, `wkwk`)
2. **`eng_markers`** - Kata-kata penanda Bahasa Inggris

**Logika Pelabelan:**
- Jika tweet mengandung kata dari **kedua** lexicon → `MIX` (Code-Switching)
- Jika lebih banyak kata Inggris → `EN`
- Selain itu → `ID`

In [ ]:
# Lexicon Bahasa Indonesia (termasuk slang dan singkatan)
indo_markers = {
    'aku', 'kamu', 'dia', 'kita', 'mereka', 'ini', 'itu', 'dan', 'atau', 'tapi',
    'yang', 'di', 'ke', 'dari', 'bisa', 'mau', 'sudah', 'udah', 'lagi', 'lg',
    'gak', 'ga', 'nggak', 'tak', 'jangan', 'sama', 'bgt', 'banget', 'dong', 'sih',
    'kok', 'deh', 'kan', 'kalo', 'kalau', 'buat', 'utk', 'untuk', 'dgn', 'dengan',
    'apa', 'kenapa', 'gimana', 'siapa', 'kapan', 'ya', 'yuk', 'wkwk', 'hehe', 
    'pake', 'pakai', 'ada', 'jadi', 'jd', 'bukan', 'krn', 'karena', 'yg', 'tidak'
}

# Lexicon Bahasa Inggris
eng_markers = {
    'i', 'you', 'he', 'she', 'we', 'they', 'it', 'this', 'that', 'and', 'or', 'but',
    'which', 'who', 'what', 'where', 'when', 'why', 'how', 'is', 'am', 'are', 'was',
    'were', 'be', 'been', 'have', 'has', 'had', 'do', 'does', 'did', 'can', 'could',
    'will', 'would', 'should', 'to', 'of', 'in', 'on', 'at', 'for', 'with', 'by',
    'from', 'about', 'just', 'like', 'so', 'not', 'no', 'yes', 'please', 'thanks',
    'my', 'your', 'actually', 'literally', 'basically', 'prefer', 'guys', 'sorry'
}

print(f"[INFO] Jumlah kata dalam lexicon Indonesia: {len(indo_markers)}")
print(f"[INFO] Jumlah kata dalam lexicon Inggris  : {len(eng_markers)}")

---
## Step 5: Fungsi Pelabelan Otomatis

Fungsi `automated_labeling()` melakukan:
1. **Preprocessing** - Ubah ke lowercase, hapus karakter non-alfabet
2. **Tokenisasi** - Pecah teks menjadi kata-kata
3. **Scoring** - Hitung berapa kata yang cocok dengan masing-masing lexicon
4. **Labeling** - Tentukan label berdasarkan skor

**Aturan Pelabelan:**

| Kondisi | Label |
|---------|-------|
| Skor ID ≥ 1 **DAN** Skor EN ≥ 1 | `MIX` (Code-Switching) |
| Skor EN > Skor ID | `EN` (English) |
| Lainnya | `ID` (Indonesia) |

In [ ]:
def automated_labeling(text):
    """
    Fungsi untuk melakukan pelabelan otomatis berdasarkan lexicon.
    
    Args:
        text: String teks tweet
    
    Returns:
        Label: 'ID', 'EN', atau 'MIX'
    """
    # Handle non-string input
    if not isinstance(text, str): 
        return 'ID'
    
    # Preprocessing: lowercase dan hapus karakter non-alfabet
    text_clean = text.lower()
    text_clean = re.sub(r'[^a-z\s]', ' ', text_clean)
    
    # Tokenisasi
    words = text_clean.split()
    if len(words) == 0: 
        return 'ID'
    
    # Hitung skor berdasarkan intersection dengan lexicon
    word_set = set(words)
    id_score = len(word_set.intersection(indo_markers))
    en_score = len(word_set.intersection(eng_markers))
    
    # Tentukan label
    if id_score >= 1 and en_score >= 1: 
        return 'MIX'  # Code-Switching terdeteksi!
    elif en_score > id_score: 
        return 'EN'
    else: 
        return 'ID'

# Contoh penggunaan
print("[TEST] Contoh hasil pelabelan:")
test_texts = [
    "Aku stuck banget sama deadline",
    "I love you so much",
    "Makan nasi goreng enak"
]
for t in test_texts:
    print(f"  '{t}' → {automated_labeling(t)}")

---
## Step 6: Terapkan Pelabelan ke Seluruh Dataset

Pada langkah ini, kita menerapkan fungsi `automated_labeling()` ke seluruh kolom `tweet` untuk menghasilkan kolom `label_bahasa`.

In [ ]:
print("[...] Sedang menjalankan algoritma pelabelan otomatis...")
df['label_bahasa'] = df['tweet'].apply(automated_labeling)
print("[✓] Pelabelan selesai!")

# Tampilkan beberapa contoh hasil
print("\n[INFO] Contoh hasil pelabelan:")
display(df[['tweet', 'label_bahasa']].head(10))

---
## Step 7: Simpan Dataset Hasil Pelabelan

Dataset yang sudah dilabeli disimpan ke file CSV untuk:
- Dokumentasi di skripsi/laporan
- Digunakan di tahap selanjutnya (training model)

In [ ]:
output_csv = 'dataset_hasil_pelabelan.csv'
df[['tweet', 'label_bahasa']].to_csv(output_csv, index=False)
print(f"[✓] File '{output_csv}' berhasil disimpan! (Lampirkan di skripsi)")

---
## Step 8: Statistik Distribusi Label

Menampilkan statistik distribusi label untuk memahami komposisi dataset.

In [ ]:
print("\n[INFO] Statistik Distribusi Label:")
print_separator()
print(f"{'KELAS BAHASA':<15} | {'JUMLAH SAMPLE':<15} | {'PERSENTASE':<15}")
print_separator()

stats = df['label_bahasa'].value_counts()
total = len(df)
for label, count in stats.items():
    print(f"{label:<15} | {count:<15} | {count/total:.2%}")
print_separator()

# Visualisasi distribusi
plt.figure(figsize=(8, 5))
stats.plot(kind='bar', color=['#3498db', '#e74c3c', '#2ecc71'])
plt.title('Distribusi Label Bahasa')
plt.xlabel('Label')
plt.ylabel('Jumlah')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

---
# TAHAP 2: PEMODELAN (NAIVE BAYES & TF-IDF)

Pada tahap ini, kita melatih model klasifikasi menggunakan:
- **TF-IDF Vectorizer** - Mengubah teks menjadi vektor numerik
- **Multinomial Naive Bayes** - Algoritma klasifikasi yang cocok untuk data teks

---
## Step 9: Text Cleaning untuk Model

Sebelum melatih model, kita melakukan preprocessing tambahan:
1. Ubah ke lowercase
2. Hapus placeholder `[username]` dan `[url]`
3. Hapus karakter non-alfanumerik

In [ ]:
print_header("TAHAP 2: PELATIHAN MODEL NAIVE BAYES")

def clean_text_final(text):
    """
    Fungsi untuk membersihkan teks sebelum diproses model.
    
    Args:
        text: String teks mentah
    
    Returns:
        String teks yang sudah dibersihkan
    """
    text = str(text).lower()
    text = re.sub(r'\[username\]|\[url\]', '', text)  # Hapus placeholder
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Hapus karakter khusus
    return text.strip()

df['text_clean'] = df['tweet'].apply(clean_text_final)
print("[✓] Text cleaning selesai!")

# Tampilkan contoh hasil cleaning
print("\n[INFO] Contoh hasil cleaning:")
display(df[['tweet', 'text_clean']].head(5))

---
## Step 10: Split Data (Training & Testing)

Dataset dibagi menjadi:
- **80% Data Latih** - Untuk melatih model
- **20% Data Uji** - Untuk mengevaluasi performa model

**Parameter penting:**
- `test_size=0.2` → 20% untuk testing
- `random_state=42` → Reproducible split
- `stratify=df['label_bahasa']` → Menjaga proporsi label di train dan test sama

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text_clean'], 
    df['label_bahasa'], 
    test_size=0.2, 
    random_state=42,
    stratify=df['label_bahasa']
)

print(f"[INFO] Data Latih : {len(X_train)} sampel")
print(f"[INFO] Data Uji   : {len(X_test)} sampel")
print(f"[INFO] Rasio      : {len(X_train)/len(df):.0%} : {len(X_test)/len(df):.0%}")

---
## Step 11: Buat dan Latih Model (Pipeline)

Kita menggunakan **Pipeline** untuk menggabungkan:
1. **TfidfVectorizer** - Mengubah teks → vektor TF-IDF
   - `ngram_range=(1, 2)` → Menggunakan unigram dan bigram
   - `max_features=5000` → Maksimal 5000 fitur
2. **MultinomialNB** - Classifier Naive Bayes
   - `alpha=0.1` → Smoothing parameter (Laplace smoothing)

In [ ]:
model = make_pipeline(
    TfidfVectorizer(ngram_range=(1, 2), max_features=5000), 
    MultinomialNB(alpha=0.1)
)

print("[...] Sedang melatih model...")
model.fit(X_train, y_train)
print("[✓] Model berhasil dilatih!")

# Info model
print("\n[INFO] Komponen Pipeline:")
for i, step in enumerate(model.steps):
    print(f"  {i+1}. {step[0]}: {type(step[1]).__name__}")

---
# TAHAP 3: EVALUASI & VISUALISASI

Pada tahap ini, kita mengevaluasi performa model menggunakan:
- **Accuracy Score** - Persentase prediksi yang benar
- **Classification Report** - Precision, Recall, F1-Score per kelas
- **Confusion Matrix** - Visualisasi kesalahan prediksi

---
## Step 12: Prediksi dan Hitung Akurasi

Model melakukan prediksi pada data uji, kemudian kita hitung akurasinya dengan rumus:

$$\text{Akurasi} = \frac{\text{Jumlah Prediksi Benar}}{\text{Total Data Uji}} \times 100\%$$

In [ ]:
print_header("TAHAP 3: EVALUASI & HASIL")

y_pred = model.predict(X_test)
acc_score = accuracy_score(y_test, y_pred)

print(f"\n{' AKURASI MODEL ':*^40}")
print(f"{acc_score:.2%}".center(40))
print("*" * 40 + "\n")

---
## Step 13: Classification Report

Laporan klasifikasi memberikan metrik detail per kelas:

| Metrik | Deskripsi |
|--------|--------|
| **Precision** | Dari semua prediksi kelas X, berapa yang benar? |
| **Recall** | Dari semua data kelas X sebenarnya, berapa yang berhasil ditemukan? |
| **F1-Score** | Harmonic mean dari Precision dan Recall |
| **Support** | Jumlah sampel per kelas di data uji |

In [ ]:
print("[INFO] Detail Laporan Klasifikasi:")
print(classification_report(y_test, y_pred))

---
## Step 14: Confusion Matrix

Confusion Matrix menunjukkan perbandingan antara **label sebenarnya** (sumbu Y) dengan **prediksi model** (sumbu X).

- Diagonal utama (warna gelap) = Prediksi benar
- Sel lainnya = Kesalahan prediksi (misclassification)

In [ ]:
try:
    cm = confusion_matrix(y_test, y_pred, labels=['ID', 'EN', 'MIX'])
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['ID', 'EN', 'MIX'], 
                yticklabels=['ID', 'EN', 'MIX'])
    plt.title('Confusion Matrix: Deteksi Code-Switching')
    plt.ylabel('Label Asli (Weak Supervision)')
    plt.xlabel('Prediksi Model')
    plt.tight_layout()
    
    # Simpan gambar
    plt.savefig('confusion_matrix.png', dpi=150)
    print("[✓] Gambar 'confusion_matrix.png' berhasil disimpan.")
    
    plt.show()
except Exception as e:
    print(f"[WARNING] Gagal membuat gambar confusion matrix: {e}")

---
## Step 15: (Opsional) Download File Hasil di Google Colab

Uncomment kode di bawah untuk mendownload file hasil ke komputer lokal.

In [ ]:
# Uncomment untuk download file hasil di Google Colab
# from google.colab import files
# files.download('dataset_hasil_pelabelan.csv')
# files.download('confusion_matrix.png')

---
## Ringkasan

Notebook ini telah menyelesaikan:

1. ✅ **Weak Supervision** - Pelabelan otomatis 100% data menggunakan lexicon
2. ✅ **Pemodelan** - Training Naive Bayes + TF-IDF
3. ✅ **Evaluasi** - Akurasi, Classification Report, Confusion Matrix

**File Output:**
- `dataset_hasil_pelabelan.csv` - Dataset berlabel untuk dokumentasi
- `confusion_matrix.png` - Visualisasi untuk skripsi/laporan